In [23]:
import os
import requests
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json


In [ ]:

model_name = "llama3.2"
llm  =   OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [38]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup =BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all("a")]
        self.links = [link for link in links if link]
        
        
    def get_contents(self):
        return f'Website Title: \n{self.title}\nWebpage Contents: \n{self.text}\n\n'

In [39]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

In [18]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [20]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [24]:
def get_links(url):
    website = Website(url)
    response = llm.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [26]:
get_links("https://www.linkedin.com/in/eddonner/")

{'links': [{'type': 'About page',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Portfolio', 'url': 'https://linkedin.com/in/eddonner/projects/'},
  {'type': 'Skills and Experience',
   'url': 'https://linkedin.com/in/eddonner/skills'}]}

In [27]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/Janus-Pro-7B',
 '/deepseek-ai/DeepSeek-V3',
 '/hexgrad/Kokoro-82M',
 '/ValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/deepseek-ai/deepseek-vl2-small',
 '/spaces/ReverseImageSearch/Reverse-Image-Search2',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cognitivecomputations/dolphin-r1',
 '/datasets/simplescaling/s1K',
 '/datasets/ServiceNow-AI/R1-Distill-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microso

In [28]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [29]:
get_links("https://www.teachinghub.in")

{'links': [{'type': 'About page',
   'url': 'https://www.teachinghub.in/about.html'},
  {'type': 'Courses page', 'url': 'https://www.teachinghub.in/courses.html'}]}

In [40]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [41]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co/'}, {'type': 'Status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Website Title: 
Hugging Face – The AI community building the future.
Webpage Contents: 
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, 

In [44]:
print(get_all_details("https://www.teachinghub.in"))

Found links: {'links': [{'type': 'About page', 'url': 'https://www.teachinghub.in/about'}, {'type': 'Courses page', 'url': 'https://www.teachinghub.in/courses'}, {'type': 'Contact page', 'url': 'https://www.teachinghub.in/contact'}]}
Landing page:
Website Title: 
Teachinghub
Webpage Contents: 
Teachinghub
Home
About
Courses
Contact
Learning Today,
Leading Tomorrow
Global learning spot
Get Started
company overview
Empowering Empowering students with affordable, in-demand IT and technology solutions, fostering a creative community, and driving sustainable education while addressing environmental challenges in a growing ecosystem. We Offering Short-term Training Sessions for Students to Explore In-Demand Topics, Promoting Diversity and Career Readiness.
Our mission is to provide accessible and inclusive learning resources to every student,
house-made woman, and unemployed individual, regardless of their background or
circumstances. We strive to bridge the gap between education and employm

In [45]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [46]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [47]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About us page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Career/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co/'}]}


ConnectionError: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12d9470d0>: Failed to resolve 'blog.huggingface.co' ([Errno 8] nodename nor servname provided, or not known)"))

In [50]:

print(get_brochure_user_prompt("teachingHUb", "https://www.teachinghub.in"))

Found links: {'links': [{'type': 'About page', 'url': 'https://www.teachinghub.in/about'}, {'type': 'Courses page', 'url': 'https://www.teachinghub.in/courses.html'}, {'type': 'Contact page', 'url': 'https://www.teachinghub.in/contact.html'}, {'type': 'Course Details page', 'url': 'https://www.teachinghub.in/course-details.html'}]}
You are looking at a company called: teachingHUb
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Website Title: 
Teachinghub
Webpage Contents: 
Teachinghub
Home
About
Courses
Contact
Learning Today,
Leading Tomorrow
Global learning spot
Get Started
company overview
Empowering Empowering students with affordable, in-demand IT and technology solutions, fostering a creative community, and driving sustainable education while addressing environmental challenges in a growing ecosystem. We Offering Short-term Training Sessions for Students to Explore In-Dema

In [ ]:
def create_brochure(company_name, url):
    response = llm.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))